# First try on transfer learning

## Cargando el dataset

Este esta ubicado en ~/.medmnist, para instalarlo correr el comando ```python -m medmnist download --size=28```

In [1]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch
import os

file_path = os.path.expanduser("~/.medmnist/breastmnist.npz")
data = np.load(file_path)

In [2]:
X_train, y_train = data['train_images'], data['train_labels']
X_val, y_val     = data['val_images'],   data['val_labels']
X_test, y_test   = data['test_images'],  data['test_labels']

In [3]:
X_train = torch.tensor(X_train, dtype=torch.float32) / 255.0
X_val   = torch.tensor(X_val, dtype=torch.float32) / 255.0
X_test  = torch.tensor(X_test, dtype=torch.float32) / 255.0

y_train = torch.tensor(y_train, dtype=torch.long)
y_val   = torch.tensor(y_val, dtype=torch.long)
y_test  = torch.tensor(y_test, dtype=torch.long)

In [4]:
if X_train.ndim == 4 and X_train.shape[3] == 1:
    X_train = X_train.permute(0, 3, 1, 2)
    X_val   = X_val.permute(0, 3, 1, 2)
    X_test  = X_test.permute(0, 3, 1, 2)
elif X_train.ndim == 3:  # si (N,H,W)
    X_train = X_train.unsqueeze(1)
    X_val   = X_val.unsqueeze(1)
    X_test  = X_test.unsqueeze(1)

print("Forma de X_train:", X_train.shape)
print("Forma de y_train:", y_train.shape)

Forma de X_train: torch.Size([546, 1, 28, 28])
Forma de y_train: torch.Size([546, 1])


In [5]:
batch_size = 64

train_dataset = TensorDataset(X_train, y_train)
val_dataset   = TensorDataset(X_val, y_val)
test_dataset  = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Creando el Modelo

In [6]:
# -------------------------
# 1️⃣ Librerías necesarias
# -------------------------
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision import models

# -------------------------
# 2️⃣ Configuración del dispositivo
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.version.hip)  # muestra la versión de ROCm
print("Usando dispositivo:", device)

# -------------------------
# 3️⃣ Cargar dataset BreastMNIST desde ~/.medmnist
# -------------------------
file_path = os.path.expanduser("~/.medmnist/breastmnist.npz")
data = np.load(file_path)
print("Arrays disponibles en el archivo:", data.files)

X_train, y_train = data['train_images'], data['train_labels']
X_val, y_val     = data['val_images'],   data['val_labels']
X_test, y_test   = data['test_images'],  data['test_labels']

# -------------------------
# 4️⃣ Convertir a tensores PyTorch y normalizar
# -------------------------
X_train = torch.tensor(X_train, dtype=torch.float32) / 255.0
X_val   = torch.tensor(X_val, dtype=torch.float32) / 255.0
X_test  = torch.tensor(X_test, dtype=torch.float32) / 255.0

y_train = torch.tensor(y_train, dtype=torch.long)
y_val   = torch.tensor(y_val, dtype=torch.long)
y_test  = torch.tensor(y_test, dtype=torch.long)

# -------------------------
# 5️⃣ Ajustar dimensiones
# -------------------------
# Convertir (N,H,W,1) a (N,C,H,W) o agregar canal si es grayscale
if X_train.ndim == 4 and X_train.shape[3] == 1:
    X_train = X_train.permute(0, 3, 1, 2)
    X_val   = X_val.permute(0, 3, 1, 2)
    X_test  = X_test.permute(0, 3, 1, 2)
elif X_train.ndim == 3:
    X_train = X_train.unsqueeze(1)
    X_val   = X_val.unsqueeze(1)
    X_test  = X_test.unsqueeze(1)

# -------------------------
# 6️⃣ Arreglar labels para CrossEntropyLoss
# -------------------------
# Squeeze para pasar de (N,1) a (N,)
y_train = y_train.squeeze()
y_val   = y_val.squeeze()
y_test  = y_test.squeeze()

# Si fueran one-hot, usar argmax
# y_train = torch.argmax(y_train, dim=1)
# y_val   = torch.argmax(y_val, dim=1)
# y_test  = torch.argmax(y_test, dim=1)

# -------------------------
# 7️⃣ Crear DataLoaders
# -------------------------
batch_size = 64

train_dataset = TensorDataset(X_train, y_train)
val_dataset   = TensorDataset(X_val, y_val)
test_dataset  = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# -------------------------
# 8️⃣ Cargar DenseNet121 preentrenado
# -------------------------
model = models.densenet121(pretrained=True)
num_classes = 2  # BreastMNIST
model.classifier = nn.Linear(model.classifier.in_features, num_classes)
model = model.to(device)

# -------------------------
# 9️⃣ Definir loss
# -------------------------
criterion = nn.CrossEntropyLoss()

# -------------------------
# 🔟 Fase 1: entrenar solo classifier
# -------------------------
# Congelar todo excepto classifier
for param in model.features.parameters():
    param.requires_grad = False

optimizer = optim.Adam(model.classifier.parameters(), lr=1e-4)
num_epochs_phase1 = 10

for epoch in range(num_epochs_phase1):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        # Redimensionar y duplicar canales
        inputs = F.interpolate(inputs, size=(224, 224), mode='bilinear', align_corners=False)
        if inputs.shape[1] == 1:
            inputs = inputs.repeat(1, 3, 1, 1)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    
    # Validación
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = F.interpolate(inputs, size=(224, 224), mode='bilinear', align_corners=False)
            if inputs.shape[1] == 1:
                inputs = inputs.repeat(1, 3, 1, 1)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    val_acc = correct / total
    print(f"[Phase 1] Epoch {epoch+1}/{num_epochs_phase1} - Loss: {epoch_loss:.4f} - Val Acc: {val_acc:.4f}")

# -------------------------
# 11️⃣ Fase 2: entrenar classifier + 10% final de la base
# -------------------------
# Descongelar ~10% final de features
all_params = list(model.features.parameters())
num_to_unfreeze = int(len(all_params) * 0.1)
for param in all_params[-num_to_unfreeze:]:
    param.requires_grad = True

# Optimizer para todos los parámetros entrenables
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
num_epochs_phase2 = 10

for epoch in range(num_epochs_phase2):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = F.interpolate(inputs, size=(224, 224), mode='bilinear', align_corners=False)
        if inputs.shape[1] == 1:
            inputs = inputs.repeat(1, 3, 1, 1)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    
    # Validación
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = F.interpolate(inputs, size=(224, 224), mode='bilinear', align_corners=False)
            if inputs.shape[1] == 1:
                inputs = inputs.repeat(1, 3, 1, 1)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    val_acc = correct / total
    print(f"[Phase 2] Epoch {epoch+1}/{num_epochs_phase2} - Loss: {epoch_loss:.4f} - Val Acc: {val_acc:.4f}")

print("Entrenamiento completo ✅")

# -------------------------
# 12️⃣ Evaluación en test
# -------------------------
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = F.interpolate(inputs, size=(224, 224), mode='bilinear', align_corners=False)
        if inputs.shape[1] == 1:
            inputs = inputs.repeat(1, 3, 1, 1)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

test_acc = correct / total
print(f"Test Accuracy: {test_acc:.4f}")


6.3.42131-fa1d09cbd
Usando dispositivo: cpu
Arrays disponibles en el archivo: ['train_images', 'val_images', 'test_images', 'train_labels', 'val_labels', 'test_labels']


/lustre/home/mgortarez/miniconda3/envs/meda_env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/lustre/home/mgortarez/miniconda3/envs/meda_env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Phase 1] Epoch 1/10 - Loss: 0.7720 - Val Acc: 0.6282
[Phase 1] Epoch 2/10 - Loss: 0.6387 - Val Acc: 0.7308
[Phase 1] Epoch 3/10 - Loss: 0.6046 - Val Acc: 0.7308
[Phase 1] Epoch 4/10 - Loss: 0.6024 - Val Acc: 0.7308


KeyboardInterrupt: 